# start to finish

## data loading

In [12]:
from typing import List
import json

def load_rumor_from_jsonl(filepath: str) -> List:
    jsons = []
    with open(filepath, encoding='utf8') as file:
        for line in file:
            jsons += [json.loads(line)]
    return jsons

In [13]:
import os

clef_path = '../clef2024-checkthat-lab/task5'
data_path = os.path.join(clef_path, 'data')
out_dir = './temp-data'

filepath_dev = os.path.join(data_path, 'English_dev.json')

dev_jsons = load_rumor_from_jsonl(filepath_dev)
print(len(dev_jsons))

32


## retrieve evidence

### naive tfidf

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from typing import List

def retrieve_relevant_documents(query: str, timeline: List):
    # Get only doc texts
    author_accounts = [t[0] for t in timeline]
    tweet_ids = [t[1] for t in timeline]
    documents = [t[2] for t in timeline]

    # Combine query and documents for TF-IDF vectorization
    combined_texts = [query] + documents
    
    # Generate TF-IDF vectors
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(combined_texts)
    
    # Calculate similarity of the query to each document
    similarity_scores = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])
    
    # Rank documents based on similarity scores
    ranked_doc_indices = similarity_scores.argsort()[0][::-1]

    # Sort the documents according to rank
    ranked_scores = [similarity_scores[0][i] for i in ranked_doc_indices]
    ranked_authors = [author_accounts[i] for i in ranked_doc_indices]
    ranked_ids = [tweet_ids[i] for i in ranked_doc_indices]
    ranked_documents = [documents[i] for i in ranked_doc_indices]

    # Create a list of tuples of shape (author, evidence_id, evidence_text, score)
    ranked_tuples = (list(zip(ranked_authors, ranked_ids, ranked_documents, ranked_scores)))
    
    return ranked_tuples

In [15]:
from tqdm import tqdm
import os
from utils import write_trec_format_output
from typing import List, Dict

def retrieve_using_tfidf(query: str, timeline: List, k: int = 5):
    """
    Parameters:
        - jsons (Dict): one rumor document from the dataset
        - k (int): top-k results to return. defaults to 5 

    Returns: 
    List of shape [(rumor_id, evidence_tweet_id, rank, score, evidence_text), ...]
    """

    ranked_docs = retrieve_relevant_documents(query, timeline)

    res = []
    for rank, (author_account, authority_tweet_id, doc_text, score) in enumerate(ranked_docs[:k]):
        res += [(author_account, authority_tweet_id, doc_text, rank+1, score)] 

    return res

In [16]:
# individual

ranked_docs_by_id = {}

for rumor_dict in dev_jsons:

    rumor_id = rumor_dict['id']
    query = rumor_dict['rumor']
    timeline = rumor_dict['timeline']

    ranked_docs_by_id[rumor_id] = []

    ranked_docs = retrieve_using_tfidf(query, timeline, 5)

    ranked_docs_by_id[rumor_id] += ranked_docs


outfile_tfidf = os.path.join(out_dir, 'tfidf-trec.txt')
write_trec_format_output(outfile_tfidf, ranked_docs_by_id, 'TFIDF-BASIC')

## fact verification

### using NLI

In [17]:
from transformers import pipeline

# Initialize the NLI pipeline with a pre-trained model
# nli_pipeline = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
nli_pipeline = pipeline("text-classification", model="roberta-large-mnli")

def check_statement_with_evidence(statement: str, evidence: str):
    # Define the candidate labels for NLI
    # candidate_labels = ["SUPPORTS", "REFUTES"]
    input_text = f"{evidence} [SEP] {statement}"

    # Use the NLI pipeline to predict the relationship
    # result = nli_pipeline(evidence, hypothesis=statement, candidate_labels=candidate_labels, multi_label=False)
    result = nli_pipeline(input_text)

    # Return the result
    return result

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
# individual

import copy

rumors_retrieved = []

for rumor_dict in dev_jsons:
    rumor = copy.deepcopy(rumor_dict) # copy explicitly so we don't mutate dev_jsons
    rumor['retrieved_evidence'] = []

    for author_account, authority_tweet_id, doc_text, rank, score in ranked_docs_by_id[rumor['id']]:
        rumor['retrieved_evidence'] += [[author_account, authority_tweet_id, doc_text, rank, score]]

    # new_dict.pop('timeline', None) # remove timeline dict key

    rumors_retrieved += [rumor]

rumors_retrieved[0]

{'id': 'AuRED_142',
 'rumor': 'Naturalization decree in preparation: Lebanese passports for sale?! https://t.co/UuQ7yMbSWJ https://t.co/Jf1K1NbZJD',
 'label': 'REFUTES',
 'timeline': [['https://twitter.com/LBpresidency',
   '1556600039211925504',
   'Today, the President of the Republic, General Michel Aoun, signed 9 laws that were previously approved by the House of Representatives. Details at the following link: https://t.co/wmrSaaEwDu'],
  ['https://twitter.com/LBpresidency',
   '1556559119045332992',
   'President Aoun received the Minister of Foreign Affairs and Expatriates, Dr. Abdullah Bouhabib, and the Minister of Social Affairs, Hector Hajjar, and discussed with them developments related to the file of displaced Syrians in Lebanon https://t.co/QLQAJSKzs1'],
  ['https://twitter.com/LBpresidency',
   '1556558220533157890',
   'President Aoun received the new British Ambassador to Lebanon, Hamish Cowell, and discussed with him Lebanese-British relations and ways to enhance them i

In [27]:
from typing import Tuple

def factcheck_using_evidence(claim: str, evidence: Tuple[str, int, str, int, int]):
    """
    Predict a judgement for a rumor using the retrieved evidence.

    Parameters:
        - rumor_dict (Dict): a Python Dict of a single rumor from the dataset, extended by the key 'rerieved_evidence'
    """
    label_map = {
        "CONTRADICTION": "REFUTES",
        "NEUTRAL": "NOT ENOUGH INFO",
        "ENTAILMENT": "SUPPORTS"
    }
    
    predicted_evidence = []
    scores = []

    for author_account, tweet_id, evidence_text, rank, score in evidence:
        res = check_statement_with_evidence(evidence_text, claim)
        label = label_map[res[0]['label']]
        score = res[0]['score']
        # CLEF CheckThat! task 5: score is [-1, +1] where 
        #   -1 means evidence strongly refuted
        #   +1 means evidence strongly supports
        if label == "REFUTES":
            score *= -1
        elif label == "NOT ENOUGH INFO":
            score = 0 # TODO uhmmm...

        predicted_evidence += [[
            author_account,
            tweet_id,
            evidence_text,
            score,
        ]]

        scores += [score]

    cumsum = sum(scores) / len(scores)
    
    if cumsum > 0.3:
        pred_label = "SUPPORTS"
    elif cumsum < -0.3:
        pred_label = "REFUTES"
    else:
        pred_label = "NOT ENOUGH INFO"

    return pred_label, predicted_evidence

In [28]:
factcheck_using_evidence(rumors_retrieved[0]['rumor'], rumors_retrieved[0]['retrieved_evidence'])

('NOT ENOUGH INFO',
 [['https://twitter.com/LBpresidency',
   '1555495801962614786',
   'President Aoun received the Lebanese Ambassador to Canada, Ambassador Fadi Ziadeh, who briefed him on the conditions of the Lebanese community in Canada, Lebanese-Canadian relations, and ways to develop them in all fields. https://t.co/HwgRPhBS5F',
   0],
  ['https://twitter.com/LBpresidency',
   '1555424541509386240',
   '“The Information Office of the Presidency of the Republic: What was published by the French newspaper “Liberation” about the “selling” of Lebanese passports to non-Lebanese is false and baseless news.',
   0],
  ['https://twitter.com/LBpresidency',
   '1555126458191855616',
   'President Aoun received the Ambassador of Finland, Tarja Fernandes, on a farewell visit on the occasion of the end of her diplomatic duties in Lebanon. He thanked her for the efforts she made to strengthen and develop Lebanese-Finnish relations, and wished her success in her new duties. https://t.co/DVIbIM

In [21]:
# individual
from utils import write_jsonlines_from_dicts
from tqdm import tqdm

res_jsons = []

for item in tqdm(rumors_retrieved):
    rumor = item['rumor']
    retrieved_evidence = item['retrieved_evidence']
    if retrieved_evidence:
        pred_label, pred_evidence = factcheck_using_evidence(rumor, retrieved_evidence)
        res_json = {
            "id": item['id'],
            "label": item['label'],
            "claim": item['rumor'],
            "predicted_label": pred_label,
            "predicted_evidence": pred_evidence,
        }
        res_jsons += [res_json]

outfile_tfidf_ver = 'temp-data/zeroshot-ver-from-tfidf.jsonl'
write_jsonlines_from_dicts(outfile_tfidf_ver, res_jsons)

100%|██████████| 32/32 [02:45<00:00,  5.18s/it]


In [35]:
from utils import write_jsonlines_from_dicts
from tqdm import tqdm

ranked_docs_by_id = {}
rumors_retrieved = []
res_jsons = []

for rumor_dict in tqdm(dev_jsons):

    # unpack the dict from the dataset
    rumor_id = rumor_dict['id']
    query = rumor_dict['rumor']
    label = rumor_dict['label']
    timeline = rumor_dict['timeline']
    evidence = rumor_dict['evidence']
    
    ranked_docs = retrieve_using_tfidf(query, timeline, 5)

    pred_label, pred_evidence = factcheck_using_evidence(query, ranked_docs)

    res_json = {
        "id": rumor_id,
        "label": label,
        "claim": query,
        "predicted_label": pred_label,
        "predicted_evidence": pred_evidence,
    }
    res_jsons += [res_json]

outfile_tfidf_ver = 'temp-data/zeroshot-ver-from-tfidf.jsonl'
write_jsonlines_from_dicts(outfile_tfidf_ver, res_jsons)

100%|██████████| 32/32 [02:34<00:00,  4.83s/it]


In [38]:
from scoring_utils import eval_run

task5_dir = '../clef2024-checkthat-lab/task5'

sample_submission_file = task5_dir + '/submission_samples/KGAT_zeroShot_verification_English_dev.json'

nli_submission_file = 'temp-data/zeroshot-ver-from-tfidf.jsonl'
ground_truth_file = task5_dir + '/data/Arabic_dev.json'
out_file = 'temp-data/out.csv'

print('sample')
eval_run(sample_submission_file,ground_truth_file, out_file)

print('nli')
eval_run(nli_submission_file,ground_truth_file, out_file)

sample
Macro_F1 0.5081585081585082
Strict Macro_F1 0.5081585081585082
nli
Macro_F1 0.23214285714285718
Strict Macro_F1 0.23214285714285718
